<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/%EA%B8%88%EA%B0%90%EC%9B%90_Dart_%EA%B8%B0%EC%97%85%EA%B3%A0%EC%9C%A0%EC%8B%9D%EB%B3%84%EB%B2%88%ED%98%B8_%EC%95%8C%EC%95%84%EB%82%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Open Dart에 있는 기업고유코드를 읽어오는 프로그램<BR>
함수 1은, 인수에 있는 단일 회사 코드를 반환하고, <BR>
함수 2는 전체 회사 코드를 반환한 다음 CSV에 저장한다.

먼저 필요한 모듈을 임포트 한다.

In [27]:
import requests
import zipfile
import pandas as pd
import io

기업 명을 입력하면, 해당하는 코드를 반환하는 함수를 정의<BR>
인수는 키와 기업명이 된다.

In [28]:
def get_company_identifier(api_key, company_name):
    base_url = "https://opendart.fss.or.kr/api/corpCode.xml"
    params = {
        "crtfc_key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # ZIP 파일을 메모리 내에서 열기
        with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
            with thezip.open('CORPCODE.xml') as thefile:
                # XML 응답을 파싱
                from xml.etree import ElementTree
                tree = ElementTree.parse(thefile)
                root = tree.getroot()

                for corp in root.findall("list"):
                    if corp.find("corp_name").text == company_name:
                        return corp.find("corp_code").text

    return None

두 번째 함수는 회사명을 받지 않고, 모든 회사를 df에 넣어 반환<BR>
인수는 API KEY만 넣어주면 된다.

In [29]:
def get_all_company_identifiers(api_key):
    base_url = "https://opendart.fss.or.kr/api/corpCode.xml"
    params = {
        "crtfc_key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # ZIP 파일을 메모리 내에서 열기
        with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
            with thezip.open('CORPCODE.xml') as thefile:
                # XML 응답을 파싱
                from xml.etree import ElementTree
                tree = ElementTree.parse(thefile)
                root = tree.getroot()

                data = []
                for corp in root.findall("list"):
                    corp_code = corp.find("corp_code").text
                    corp_name = corp.find("corp_name").text
                    stock_code = corp.find("stock_code").text
                    modify_date = corp.find("modify_date").text
                    data.append({
                        'corp_code': corp_code,
                        'corp_name': corp_name,
                        'stock_code': stock_code,
                        'modify_date': modify_date
                    })

                # 데이터프레임으로 변환
                df = pd.DataFrame(data)
                return df

    return None

키를 설정하고, 호출하고자 하는 회사명을 기입한다.

In [30]:
# 사용 예시
api_key = "a06ed5ecaee6fac7168de666b7912090bfd1af23"
company_name = "신한은행"

첫번째 함수를 호출하여 회사 코드를 읽어 온다.굵은 텍스트

In [31]:
company_code = get_company_identifier(api_key, company_name)

읽어온 코드를 출력한다.

In [ ]:
if company_code:
    print(f"{company_name}의 고유식별번호: {company_code}")
else:
    print(f"{company_name}의 고유식별번호를 찾을 수 없습니다.")


이번에는 두 번째 함수를 호출하여, 모든 식별자를 받아 온다.

In [ ]:
df = get_all_company_identifiers(api_key)

if df is not None:
    print(df)
else:
    print("기업 식별번호를 불러올 수 없습니다.")

모든 식별자를 별도의 CSV 파일에 저장한다.<BR>마지막에 반드시 UTF-8-sig를 명기해야 한다.<BR> UTF-8로 인코딩한다는 식별자에 해당한다.

In [35]:
df.to_csv("all_company_identifiers.csv", index=False, encoding='utf-8-sig')